- [x] Using the titanic data, in your classification-exercises repository, create a notebook, decision_tree.ipynb where you will do the following:


In [2]:
# Import full libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import personal modules
import acquire as a
import prepare as p
import model as m

# Selective imports
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Handle warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Load titanic
titanic_raw = a.get_titanic_data()
titanic_raw.head()

Reading from file...


,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [4]:
# Prepare titanic
titanic_prepped = p.prep_titanic(titanic_raw)
titanic_prepped.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,Southampton,1


In [5]:
# make sure dataset is fully clean
a.df_info(titanic_prepped)

,nunique,dtypes,isnull,sample
passenger_id,891,object,0,284
survived,2,object,0,0
pclass,3,object,0,1
sex,2,object,0,male
age,91,float64,0,38.23
sibsp,7,object,0,0
parch,7,object,0,0
fare,248,float64,0,26.0
embark_town,3,object,0,Southampton
alone,2,object,0,1


> At this time, I've moved a piece of the function from the preprocess titanic function over to the prep_titanic function to handle the age nulls.

> It's still missing the piece to fix the nulls in embark_town.

In [7]:
# fill based on mode
titanic_prepped['embark_town'] = titanic_prepped.embark_town.fillna(titanic_prepped.embark_town.mode()[0])

In [8]:
# check the data again
a.df_info(titanic_prepped)

,nunique,dtypes,isnull,sample
passenger_id,891,object,0,562
survived,2,object,0,0
pclass,3,object,0,2
sex,2,object,0,male
age,91,float64,0,28.0
sibsp,7,object,0,0
parch,7,object,0,0
fare,248,float64,0,13.5
embark_town,3,object,0,Southampton
alone,2,object,0,1


In [9]:
# Split titanic
train_titanic,validate_titanic,test_titanic = p.split_df(titanic_prepped,'survived')
train_titanic.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embark_town,alone
776,776,0,3,male,25.14,0,0,7.7500,Queenstown,1
829,829,1,1,female,62.00,0,0,80.0000,Southampton,1
215,215,1,1,female,31.00,1,0,113.2750,Cherbourg,0
258,258,1,1,female,35.00,0,0,512.3292,Cherbourg,1
129,129,0,3,male,45.00,0,0,6.9750,Southampton,1


> In the future, I want to split the dataset before applying the preparations.

1. [x] What is your baseline prediction? What is your baseline accuracy? *remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.*

In [12]:
# Get baseline prediction
base_pred = train_titanic.survived.value_counts().index[0]
base_pred

0

In [13]:
# Get baseline accuracy
base_acc = (train_titanic.survived == base_pred).mean()
base_acc

0.6161048689138576

2. [X] Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [15]:
# Preprocess data
encoded_train,encoded_val,encoded_test = [m.preprocess_titanic(df) for df in [train_titanic,validate_titanic,test_titanic]]
encoded_train

,survived,pclass,age,sibsp,parch,fare,alone,is_male,is_Queenstown,is_Southampton
776,0.0,3.0,25.14,0.0,0.0,7.7500,1.0,1.0,1.0,0.0
829,1.0,1.0,62.00,0.0,0.0,80.0000,1.0,0.0,0.0,1.0
215,1.0,1.0,31.00,1.0,0.0,113.2750,0.0,0.0,0.0,0.0
258,1.0,1.0,35.00,0.0,0.0,512.3292,1.0,0.0,0.0,0.0
129,0.0,3.0,45.00,0.0,0.0,6.9750,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
125,1.0,3.0,12.00,1.0,0.0,11.2417,0.0,1.0,0.0,0.0
360,0.0,3.0,40.00,1.0,4.0,27.9000,0.0,1.0,0.0,1.0
55,1.0,1.0,38.23,0.0,0.0,35.5000,1.0,1.0,0.0,1.0
298,1.0,1.0,38.23,0.0,0.0,30.5000,1.0,1.0,0.0,1.0


In [16]:
# get the X's
X_train,X_validate,X_test = [encoded.drop(columns='survived') for encoded in [encoded_train,encoded_val,encoded_test]]
X_train

,pclass,age,sibsp,parch,fare,alone,is_male,is_Queenstown,is_Southampton
776,3.0,25.14,0.0,0.0,7.7500,1.0,1.0,1.0,0.0
829,1.0,62.00,0.0,0.0,80.0000,1.0,0.0,0.0,1.0
215,1.0,31.00,1.0,0.0,113.2750,0.0,0.0,0.0,0.0
258,1.0,35.00,0.0,0.0,512.3292,1.0,0.0,0.0,0.0
129,3.0,45.00,0.0,0.0,6.9750,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
125,3.0,12.00,1.0,0.0,11.2417,0.0,1.0,0.0,0.0
360,3.0,40.00,1.0,4.0,27.9000,0.0,1.0,0.0,1.0
55,1.0,38.23,0.0,0.0,35.5000,1.0,1.0,0.0,1.0
298,1.0,38.23,0.0,0.0,30.5000,1.0,1.0,0.0,1.0


In [17]:
# get the y's
y_train,y_validate,y_test = [encoded.survived for encoded in [encoded_train,encoded_val,encoded_test]]
y_train

776    0.0
829    1.0
215    1.0
258    1.0
129    0.0
      ... 
125    1.0
360    0.0
55     1.0
298    1.0
768    0.0
Name: survived, Length: 534, dtype: float64

In [18]:
# create the model
clf1 = DecisionTreeClassifier(max_depth=3)
clf1

DecisionTreeClassifier(max_depth=3)

In [19]:
# fit the model
clf1.fit(X_train, y_train)
clf1

DecisionTreeClassifier(max_depth=3)

In [20]:
# make prediction
clf1.predict(X_train)[:10]

array([0., 1., 1., 1., 0., 0., 1., 0., 1., 0.])

In [21]:
clf1.score(X_train,y_train)

0.8445692883895131

3. [x] Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [23]:
# Evaluate model score
clf1.score(X_train,y_train)

0.8445692883895131

In [24]:
# Run confusion matrix
pd.DataFrame(confusion_matrix(y_train,clf1.predict(X_train)))

,0,1
0,300,29
1,54,151


In [25]:
# Run classification report
print(classification_report(y_train,clf1.predict(X_train)))

              precision    recall  f1-score   support

         0.0       0.85      0.91      0.88       329
         1.0       0.84      0.74      0.78       205

    accuracy                           0.84       534
   macro avg       0.84      0.82      0.83       534
weighted avg       0.84      0.84      0.84       534



4. [x] Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [27]:
# Get accuracy, precision, recall, f1-score, and support for train
print(classification_report(y_train,clf1.predict(X_train)))

              precision    recall  f1-score   support

         0.0       0.85      0.91      0.88       329
         1.0       0.84      0.74      0.78       205

    accuracy                           0.84       534
   macro avg       0.84      0.82      0.83       534
weighted avg       0.84      0.84      0.84       534



In [28]:
pd.DataFrame(confusion_matrix(y_train,clf1.predict(X_train),normalize='true'))

,0,1
0,0.911854,0.088146
1,0.263415,0.736585


5. [x] Run through steps 2-4 using a different `max_depth` value.

In [30]:
# choose depth of 6
clf2 = DecisionTreeClassifier(max_depth=6)
clf2

DecisionTreeClassifier(max_depth=6)

In [31]:
# choose depth of 9
clf3 = DecisionTreeClassifier(max_depth=9)
clf3

DecisionTreeClassifier(max_depth=9)

In [32]:
# choose depth of 12
clf4 = DecisionTreeClassifier(max_depth=12)
clf4

DecisionTreeClassifier(max_depth=12)

In [33]:
# choose depth of 15
clf5 = DecisionTreeClassifier(max_depth=15)
clf5

DecisionTreeClassifier(max_depth=15)

In [122]:
# test out making a dictionary of clfs
clf_dict = {
    'clf1':clf1,
    'clf2':clf2,
    'clf3':clf3,
    'clf4':clf4,
    'clf5':clf5,
}

In [35]:
# test out calling the models
for clf in clf_dict:
    print(clf_dict[clf])

DecisionTreeClassifier(max_depth=3)
DecisionTreeClassifier(max_depth=6)
DecisionTreeClassifier(max_depth=9)
DecisionTreeClassifier(max_depth=12)
DecisionTreeClassifier(max_depth=15)


In [73]:
# fit models using this technique
for clf in clf_dict:
    clf_dict[clf].fit(X_train,y_train)

6. [ ] Which model performs better on your in-sample data?

In [79]:
# print out scores per clf
for clf in clf_dict:
    print(f'{clf} score: {clf_dict[clf].score(X_train,y_train)}')

clf1 score: 0.8445692883895131
clf2 score: 0.8876404494382022
clf3 score: 0.947565543071161
clf4 score: 0.9662921348314607
clf5 score: 0.9831460674157303


In [88]:
# print out classification reports
for clf in clf_dict:
    print(f'{clf}:\n{classification_report(y_train,clf_dict[clf].predict(X_train))}\n\n')

clf1:
              precision    recall  f1-score   support

         0.0       0.85      0.91      0.88       329
         1.0       0.84      0.74      0.78       205

    accuracy                           0.84       534
   macro avg       0.84      0.82      0.83       534
weighted avg       0.84      0.84      0.84       534



clf2:
              precision    recall  f1-score   support

         0.0       0.91      0.91      0.91       329
         1.0       0.86      0.85      0.85       205

    accuracy                           0.89       534
   macro avg       0.88      0.88      0.88       534
weighted avg       0.89      0.89      0.89       534



clf3:
              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96       329
         1.0       0.98      0.88      0.93       205

    accuracy                           0.95       534
   macro avg       0.95      0.94      0.94       534
weighted avg       0.95      0.95      0.95       53

7. [x] Which model performs best on your out-of-sample data, the `validate` set?


In [93]:
# Iterate through models to get accuracy score of each
for clf in clf_dict:
    # clf1.score(X_validate,y_validate)
    print(f'{clf} validate score: {clf_dict[clf].score(X_validate,y_validate)}')

clf1 validate score: 0.7921348314606742
clf2 validate score: 0.7696629213483146
clf3 validate score: 0.7752808988764045
clf4 validate score: 0.7696629213483146
clf5 validate score: 0.7471910112359551


In [95]:
for clf in clf_dict:
    print(f'{clf}:\n{classification_report(y_validate,clf_dict[clf].predict(X_validate))}\n\n')

clf1:
              precision    recall  f1-score   support

         0.0       0.81      0.87      0.84       110
         1.0       0.76      0.66      0.71        68

    accuracy                           0.79       178
   macro avg       0.78      0.77      0.77       178
weighted avg       0.79      0.79      0.79       178



clf2:
              precision    recall  f1-score   support

         0.0       0.82      0.80      0.81       110
         1.0       0.69      0.72      0.71        68

    accuracy                           0.77       178
   macro avg       0.76      0.76      0.76       178
weighted avg       0.77      0.77      0.77       178



clf3:
              precision    recall  f1-score   support

         0.0       0.81      0.83      0.82       110
         1.0       0.71      0.69      0.70        68

    accuracy                           0.78       178
   macro avg       0.76      0.76      0.76       178
weighted avg       0.77      0.78      0.77       17

> Based on what I see, it looks like model 1 fares the best overall in terms of accuracy, although depending on your goal of the data, model 2 may be a close second.

In [131]:
# Run the test!
clf1.score(X_test,y_test)

0.776536312849162

---

1. [ ] Work through these same exercises using the Telco dataset.

In [99]:
# import Telco

In [101]:
# examine Telco 

In [105]:
# Prepare Telco

In [103]:
# split Telco

In [ ]:
# 

2. [ ] Experiment with this model on other datasets with a higher number of output classes.